In [101]:
import requests
import lxml.html as lh
 
gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
# print(page)
doc = lh.fromstring(page.content)
# print(type(doc))
link_list = doc.xpath("//*/ul/li/a/@href")
# print(link_list)

file_list = [x for x in link_list if str.isdigit(x[0:4]) and int(x[0:4])>2005 if 201808<=int(x[0:6])<=201810] #ambil tahun 2018 3 bln ajah
# print(file_list)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib.request as urllib
import zipfile
import glob
import operator

local_path = 'D:/Digitalent Kominfo 2018/GDELT/'

fips_country_code = 'ID'

# for compressed_file in file_list[1:2]:
for compressed_file in file_list[infilecounter:]:
    print(compressed_file)
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, '),
        urllib.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
        
    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')
    
    # parse each of the csv files in the working directory,
    print('parsing,'),
    for infile_name in glob.glob(local_path + 'tmp/*'):
        outfile_name = local_path + 'country/' + fips_country_code + '%04i.tsv' % outfilecounter

        # open the infile and outfile
        with open(infile_name, mode='r', encoding='latin1') as infile, open(outfile_name, mode='w', encoding='latin1') as outfile:
            outfile.write('GLOBALEVENTID	SQLDATE	MonthYear	Year	FractionDate	Actor1Code	Actor1Name	Actor1CountryCode	Actor1KnownGroupCode	Actor1EthnicCode	Actor1Religion1Code	Actor1Religion2Code	Actor1Type1Code	Actor1Type2Code	Actor1Type3Code	Actor2Code	Actor2Name	Actor2CountryCode	Actor2KnownGroupCode	Actor2EthnicCode	Actor2Religion1Code	Actor2Religion2Code	Actor2Type1Code	Actor2Type2Code	Actor2Type3Code	IsRootEvent	EventCode	EventBaseCode	EventRootCode	QuadClass	GoldsteinScale	NumMentions	NumSources	NumArticles	AvgTone	Actor1Geo_Type	Actor1Geo_FullName	Actor1Geo_CountryCode	Actor1Geo_ADM1Code	Actor1Geo_Lat	Actor1Geo_Long	Actor1Geo_FeatureID	Actor2Geo_Type	Actor2Geo_FullName	Actor2Geo_CountryCode	Actor2Geo_ADM1Code	Actor2Geo_Lat	Actor2Geo_Long	Actor2Geo_FeatureID	ActionGeo_Type	ActionGeo_FullName	ActionGeo_CountryCode	ActionGeo_ADM1Code	ActionGeo_Lat	ActionGeo_Long	ActionGeo_FeatureID	DATEADDED	SOURCEURL\n')
            for line in infile:
#                 print(line.split('\t'))
#                 # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
        
    infilecounter += 1
    print('done')

20181031.export.CSV.zip
extracting,
parsing,
done
20181030.export.CSV.zip
extracting,
parsing,
done
20181029.export.CSV.zip
extracting,
parsing,
done
20181028.export.CSV.zip
extracting,
parsing,
done
20181027.export.CSV.zip
extracting,
parsing,
done
20181026.export.CSV.zip
extracting,
parsing,
done
20181025.export.CSV.zip
extracting,
parsing,
done
20181024.export.CSV.zip
extracting,
parsing,
done
20181023.export.CSV.zip
extracting,
parsing,
done
20181022.export.CSV.zip
extracting,
parsing,
done
20181021.export.CSV.zip
extracting,
parsing,
done
20181020.export.CSV.zip
extracting,
parsing,
done
20181019.export.CSV.zip
extracting,
parsing,
done
20181018.export.CSV.zip
extracting,
parsing,
done
20181017.export.CSV.zip
extracting,
parsing,
done
20181016.export.CSV.zip
extracting,
parsing,
done
20181015.export.CSV.zip
extracting,
parsing,
done
20181014.export.CSV.zip
extracting,
parsing,
done
20181013.export.CSV.zip
extracting,
parsing,
done
20181012.export.CSV.zip
extracting,
parsing,
done


In [102]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = "3306"
username = 'root'
password = ''
database = 'gdelt'

engine=create_engine('mysql+pymysql://'+username+':'+password+'@'+host+':'+port+'/'+database+'?charset=latin1')
# create_engine('mysql+pymysql://root:@localhost:3306/classicmodels')

def run(sql):
    df=pd.read_sql_query(sql, engine)
    return df

In [104]:
for countryfile in glob.glob(local_path+'country/*'):
    print(countryfile)
    df=pd.read_csv(countryfile,delimiter='\t',encoding='latin1', header=0)
    df.to_sql('gdelt_content',con=engine,if_exists='append',index=False)

D:/Digitalent Kominfo 2018/GDELT/country\ID0000.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0001.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0002.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0003.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0004.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0005.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0006.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0007.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0008.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0009.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0010.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0011.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0012.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0013.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0014.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0015.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0016.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0017.tsv
D:/Digitalent Kominfo 2018/GDELT/country\ID0018.tsv
D:/Digitalen

In [132]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]
print(offset,limit)

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='gdelt',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}

with connection.cursor() as cursor:
#     sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') LIMIT "+str(offset)+","+str(limit)
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') limit 10"
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
#             r["SOURCEURL"] = r["SOURCEURL"].split('<UDIV>')
            print(r["SOURCEURL"])
            article = Article(r["SOURCEURL"])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

-f C:\Users\ivan\AppData\Roaming\jupyter\runtime\kernel-f5757c64-f775-477d-b95e-69bebd371864.json
SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') limit 10
http://www.keranews.org/post/myanmars-military-leaders-should-be-tried-genocide-un-investigators-say
http://www.keranews.org/post/myanmars-military-leaders-should-be-tried-genocide-un-investigators-say
http://www.keranews.org/post/myanmars-military-leaders-should-be-tried-genocide-un-investigators-say
http://www.keranews.org/post/myanmars-military-leaders-should-be-tried-genocide-un-investigators-say
http://www.keranews.org/post/myanmars-military-leaders-should-be-tried-genocide-un-investigators-say
http://www.keranews.org/post/myanmars-military-leaders-should-be-tried-genocide-un-investigators-say
https://www.onenewspage.com/n/Politics/1zj9d2cd09/How-Recent-Shootings-In-Florida-Are-Affecting-Politics.htm
https://www.cqnews.com.au/news/local-takes-asylum-seekers-crabbing-after-he-finds/3504944/
https://www.cqn

# DI BAWAH ASLI CODE PAK GUNTUR!!!!!

# Download

Berikut adalah contoh script untuk mengunduh data GDELT dengan country code ID. Selanjutnya dari csv bisa dipindah ke mysql

In [ ]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits
file_list = [x for x in link_list if str.isdigit(x[0:4])]
print(file_list)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'E:/GDELT/'

fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(compressed_file)

    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, '),
        urllib.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)

    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')

    # parse each of the csv files in the working directory,
    print('parsing,'),
    for infile_name in glob.glob(local_path + 'tmp/*'):
        outfile_name = local_path + 'country/' + fips_country_code + '%04i.tsv' % outfilecounter

        # open the infile and outfile
        with open(infile_name, mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
    infilecounter += 1
    print('done')

# Scraping GDELT
Berikut adalah kode contoh untuk melakukan scraping dengan library newspaper. Skrip ini akan berjalan jika data dari GDELT telah tersimpan ke MySQL

In [ ]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}

with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') LIMIT "+str(offset)+","+str(limit)
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r["URL"] = r["URL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

Berikut adalah contoh untuk melakukan pickling data GDELT

In [ ]:
import glob
import pandas as pd

local_path = 'E:/GDELT/'
fips_country_code = 'ID'
# Get the GDELT field names from a helper file
colnames = pd.read_excel('CSV.header.fieldids.xlsx', sheetname='Sheet1',
                         index_col='Column ID', parse_cols=1)['Field Name']

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path + 'country/' + fips_country_code + '*')
DFlist = []
for active_file in files:
    print active_file
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))
    break

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path + 'backupsmall' + fips_country_code + '.pickle')

# once everythin is safely stored away, remove the temporary files
# for active_file in files:
#     os.remove(active_file)